### 1. Download the Dataset
Using scikit-learn's Built-in Dataset:      

The Breast Cancer Wisconsin Diagnostic dataset is available in scikit-learn. This eliminates the need to download it manually.

In [1]:
!pip install pandas
!pip install scikit-learn
!pip install joblib
!pip install streamlit
!pip install numpy
!pip install warnings


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [2]:
!python.exe -m pip install --upgrade pip

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import warnings
from sklearn.exceptions import ConvergenceWarning

# Optionally suppress convergence warnings (not recommended)
# warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

# Check for missing values
print("Missing values in each feature:\n", X.isnull().sum())

# Feature Selection
selector = SelectKBest(score_func=f_classif, k=10)
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())

# Save the selected features
joblib.dump(selected_features, 'selected_features.pkl')
print("Selected features saved to 'selected_features.pkl'.")

# Select the columns based on feature selection
X_selected_df = pd.DataFrame(X_selected, columns=selected_features)

# Standardize Features (only selected features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected_df)
X_scaled_df = pd.DataFrame(X_scaled, columns=selected_features)

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')
print("Scaler saved to 'scaler.pkl'.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, random_state=42
)

# Define parameter grid for Grid Search
parameter_space = {
    'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'lbfgs'],  # Added 'lbfgs'
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

mlp = MLPClassifier(
    max_iter=500,  # Increased from 100 to 500
    random_state=42
)

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5, verbose=2)
clf.fit(X_train, y_train)

# Best parameters
print('Best parameters found:\n', clf.best_params_)

# Train the final model with best parameters
best_mlp = clf.best_estimator_
best_mlp.fit(X_train, y_train)

# Evaluate the model
y_pred = best_mlp.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the trained model
joblib.dump(best_mlp, 'mlp_model.pkl')
print("Trained model saved to 'mlp_model.pkl'.")


Missing values in each feature:
 mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
dtype: int64
Selected Features: ['mean radius', 'mean perimeter', 'mean area', 'mean concavity', 